In [54]:
import pandas as pd 
import ast
from sklearn.preprocessing import MinMaxScaler
import os
import warnings 
warnings.filterwarnings('ignore')


os.chdir('C:/Users/shara/Projects/2025_Siam-ML-Hack')

In [ ]:

def add_feature(row):
    base_path = "src/raw_data"
    
    file_name = row.iloc[0]
    recovery_range = ast.literal_eval(row.iloc[1])
    drop_range = ast.literal_eval(row.iloc[2])

    train_path = os.path.join(base_path, "train", file_name)
    test_path = os.path.join(base_path, "test", file_name)

    if os.path.exists(train_path):
        file_path = train_path
    else: 
        file_path = test_path

    df = pd.read_csv(file_path, names=["time", "pressure"], header=None, sep="\t")

    scaler = MinMaxScaler()
    df[['time', 'pressure']] = scaler.fit_transform(df[['time', 'pressure']])

    df['recovery_flag'] = 0
    df['drop_flag'] = 0

    for start, end in recovery_range:
        norm_start, norm_end = scaler.transform([[start, df['pressure'].min()]])[0][0], scaler.transform([[end, df['pressure'].min()]])[0][0]
        df.loc[(df['time'] >= norm_start) & (df['time'] <= norm_end), 'recovery_flag'] = 1

    for start, end in drop_range:
        norm_start, norm_end = scaler.transform([[start, df['pressure'].min()]])[0][0], scaler.transform([[end, df['pressure'].min()]])[0][0]
        df.loc[(df['time'] >= norm_start) & (df['time'] <= norm_end), 'drop_flag'] = 1

    # print(df)
    return df


ground_truth = pd.read_csv('src/raw_data/ground_truth.csv')
result_df = add_feature(ground_truth.iloc[0])
for i in range(1, ground_truth.shape[0]):
    row = ground_truth.iloc[i]
    changed_df = add_feature(row)
    result_df = pd.concat([result_df, changed_df], ignore_index=True)

result_df

           time  pressure  recovery_flag  drop_flag
0      0.000000  0.297505              0          0
1      0.000003  0.297505              0          0
2      0.000055  0.297931              0          0
3      0.000058  0.297931              0          0
4      0.000125  0.297931              0          0
...         ...       ...            ...        ...
30997  0.999915  0.108552              0          0
30998  0.999953  0.108445              0          0
30999  0.999957  0.108445              0          0
31000  0.999986  0.108445              0          0
31001  1.000000  0.107379              0          0

[31002 rows x 4 columns]
           time  pressure  recovery_flag  drop_flag
0      0.000000  0.084746              0          0
1      0.000002  0.084746              0          0
2      0.000008  0.084746              0          0
3      0.000008  0.084746              0          0
4      0.000026  0.084746              0          0
...         ...       ...            .

,time,pressure,recovery_flag,drop_flag
0,0.000000,0.297505,0,0
1,0.000003,0.297505,0,0
2,0.000055,0.297931,0,0
3,0.000058,0.297931,0,0
4,0.000125,0.297931,0,0
...,...,...,...,...
1946291,0.999839,0.113267,0,0
1946292,0.999869,0.113267,0,0
1946293,0.999889,0.113267,0,0
1946294,0.999976,0.113267,0,0
